## Load Data

In [1]:
dataRaw = readlines("caveNetwork.txt")
caveLinks = split.(dataRaw,'-')

19-element Array{Array{SubString{String},1},1}:
 ["lg", "GW"]
 ["pt", "start"]
 ["pt", "uq"]
 ["nx", "lg"]
 ["ve", "GW"]
 ["start", "nx"]
 ["GW", "start"]
 ["GW", "nx"]
 ["pt", "SM"]
 ["sx", "GW"]
 ["lg", "end"]
 ["nx", "SM"]
 ["lg", "SM"]
 ["pt", "nx"]
 ["end", "ve"]
 ["ve", "SM"]
 ["TG", "uq"]
 ["end", "SM"]
 ["SM", "uq"]

In [2]:
dataSmallTest = "start-A
start-b
A-c
A-b
b-d
A-end
b-end"
smallCaveLinks = split.(split(dataSmallTest,"\n"),"-")

7-element Array{Array{SubString{String},1},1}:
 ["start", "A"]
 ["start", "b"]
 ["A", "c"]
 ["A", "b"]
 ["b", "d"]
 ["A", "end"]
 ["b", "end"]

## Part 1

In [95]:
function buildCaveNetwork(caveLinks)
    caves = [string(x) for x in Set(hcat(caveLinks...))]
    caveConnects = Array[]
    for cave in caves
        neighborCaveList = String[]
        neighborCaves = findall(x->(x[1] == cave || x[2] == cave),caveLinks)
        for i in neighborCaves
            if caveLinks[i][1] == cave
                push!(neighborCaveList,string(caveLinks[i][2]))
            else
                push!(neighborCaveList,string(caveLinks[i][1]))
            end
        end
        push!(caveConnects, neighborCaveList)
    end
    return caves,caveConnects
end

function traverseCaves(cave::String,
                    allCaves::Array{String,1},
                    caveLinkList::Array{Array,1},
                    path::Array{String,1})
    
    push!(path,cave)
    #print(cave," ")
    caveNeigbors = caveLinkList[findfirst(x -> x==cave,allCaves)]
    returnPath = 0
    for neighbor in caveNeigbors
        if all(isuppercase,neighbor)
            returnPath += traverseCaves(neighbor,allCaves,caveLinkList,copy(path))
        else
            if neighbor == "end"
                returnPath += 1 #
                println(vcat(path,"end")) # exit found
            elseif !(neighbor in path)
                returnPath += traverseCaves(neighbor,allCaves,caveLinkList,copy(path))
            #else
                #returnPath = nothing # dead end..
            end
        end
    end
    return returnPath
end

traverseCaves (generic function with 1 method)

In [4]:
cavesSTest,linksSTest = buildCaveNetwork(smallCaveLinks)

(["A", "c", "start", "b", "end", "d"], Array[["start", "c", "b", "end"], ["A"], ["A", "b"], ["start", "A", "d", "end"], ["A", "b"], ["b"]])

In [11]:
traverseCaves("start",cavesSTest,linksSTest,String[])

["start", "A", "c", "A", "b", "A", "end"]
["start", "A", "c", "A", "b", "end"]
["start", "A", "c", "A", "end"]
["start", "A", "b", "A", "c", "A", "end"]
["start", "A", "b", "A", "end"]
["start", "A", "b", "end"]
["start", "A", "end"]
["start", "b", "A", "c", "A", "end"]
["start", "b", "A", "end"]
["start", "b", "end"]


10

In [6]:
caves,links = buildCaveNetwork(caveLinks)

(["SM", "pt", "uq", "ve", "nx", "sx", "start", "TG", "GW", "end", "lg"], Array[["pt", "nx", "lg", "ve", "end", "uq"], ["start", "uq", "SM", "nx"], ["pt", "TG", "SM"], ["GW", "end", "SM"], ["lg", "start", "GW", "SM", "pt"], ["GW"], ["pt", "nx", "GW"], ["uq"], ["lg", "ve", "start", "nx", "sx"], ["lg", "ve", "SM"], ["GW", "nx", "end", "SM"]])

In [7]:
traverseCaves("start",caves,links,String[])

3708

## Part 2

In [92]:
function travelCaves(cave::String,
                    allCaves::Array{String,1},
                    caveLinkList::Array{Array,1},
                    path::Array{String,1})
    push!(path,cave)
    caveNeigbors = caveLinkList[findfirst(x -> x==cave,allCaves)]
    returnPath = 0
    for neighbor in caveNeigbors
        if all(isuppercase,neighbor) #always enter uppercase cave 
            returnPath += travelCaves(neighbor,allCaves,caveLinkList,copy(path)) 
        else
            # exit found, terminate search and print path
            if neighbor == "end" 
                returnPath += 1  
                #println(vcat(path,"end"))
            #enter *one* lowercase(lc) cave at most twice, other lc caves at most once, and start only once at beginning
            elseif neighbor != "start"
                #check if *any* lc cave has been visitied twice already
                lcCavesVisited = path[findall(x->any(islowercase,x)==true,path)]
                uniqueLcs = unique(lcCavesVisited)
                numVisitsLc = zeros(Int,length(uniqueLcs))
                for (i,lcCave) in enumerate(uniqueLcs)
                    numVisitsLc[i]=count(x->x == lcCave,lcCavesVisited)
                end
                
                if !( any(numVisitsLc.>=2) && (neighbor in path) )
                    returnPath += travelCaves(neighbor,allCaves,caveLinkList,copy(path))
                end
            # else dead end..
            end
        end
    end
    return returnPath
end

travelCaves (generic function with 1 method)

In [93]:
travelCaves("start",cavesSTest,linksSTest,String[])

36

In [94]:
travelCaves("start",caves,links,String[])

93858